***GENERATED CODE FOR c PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Argentina', 'max': 'USA', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PhysicalActivityLevel', 'transformation_label': 'String Indexer'}], 'feature': 'PhysicalActivityLevel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PhysicalActivityLevel'}, {'feature_label': 'PhysicalActivityLevel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PhysicalActivityLevel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SmokingStatus', 'transformation_label': 'String Indexer'}], 'feature': 'SmokingStatus', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Current', 'max': 'Never', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SmokingStatus'}, {'feature_label': 'SmokingStatus', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SmokingStatus')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AlcoholConsumption', 'transformation_label': 'String Indexer'}], 'feature': 'AlcoholConsumption', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Never', 'max': 'Regularly', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AlcoholConsumption'}, {'feature_label': 'AlcoholConsumption', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AlcoholConsumption')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diabetes', 'transformation_label': 'String Indexer'}], 'feature': 'Diabetes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Diabetes'}, {'feature_label': 'Diabetes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Diabetes')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Hypertension', 'transformation_label': 'String Indexer'}], 'feature': 'Hypertension', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Hypertension'}, {'feature_label': 'Hypertension', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Hypertension')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CholesterolLevel', 'transformation_label': 'String Indexer'}], 'feature': 'CholesterolLevel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Normal', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CholesterolLevel'}, {'feature_label': 'CholesterolLevel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CholesterolLevel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FamilyHistoryofAlzheimers', 'transformation_label': 'String Indexer'}], 'feature': 'FamilyHistoryofAlzheimers', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FamilyHistoryofAlzheimers'}, {'feature_label': 'FamilyHistoryofAlzheimers', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'FamilyHistoryofAlzheimers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepressionLevel', 'transformation_label': 'String Indexer'}], 'feature': 'DepressionLevel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepressionLevel'}, {'feature_label': 'DepressionLevel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepressionLevel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SleepQuality', 'transformation_label': 'String Indexer'}], 'feature': 'SleepQuality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Poor', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SleepQuality'}, {'feature_label': 'SleepQuality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SleepQuality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DietaryHabits', 'transformation_label': 'String Indexer'}], 'feature': 'DietaryHabits', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Unhealthy', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DietaryHabits'}, {'feature_label': 'DietaryHabits', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DietaryHabits')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AirPollutionExposure', 'transformation_label': 'String Indexer'}], 'feature': 'AirPollutionExposure', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AirPollutionExposure'}, {'feature_label': 'AirPollutionExposure', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AirPollutionExposure')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MaritalStatus', 'transformation_label': 'String Indexer'}], 'feature': 'MaritalStatus', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Married', 'max': 'Widowed', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MaritalStatus'}, {'feature_label': 'MaritalStatus', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MaritalStatus')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GeneticRiskFactor', 'transformation_label': 'String Indexer'}], 'feature': 'GeneticRiskFactor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GeneticRiskFactor'}, {'feature_label': 'GeneticRiskFactor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GeneticRiskFactor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SocialEngagementLevel', 'transformation_label': 'String Indexer'}], 'feature': 'SocialEngagementLevel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SocialEngagementLevel'}, {'feature_label': 'SocialEngagementLevel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SocialEngagementLevel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StressLevels', 'transformation_label': 'String Indexer'}], 'feature': 'StressLevels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StressLevels'}, {'feature_label': 'StressLevels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StressLevels')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AlzheimersDiagnosis', 'transformation_label': 'String Indexer'}], 'feature': 'AlzheimersDiagnosis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AlzheimersDiagnosis'}, {'feature_label': 'AlzheimersDiagnosis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AlzheimersDiagnosis')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run cHooks.ipynb
try:
	#sourcePreExecutionHook()

	alzhimersdataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Alzhimers dataset.csv', 'filename': 'Alzhimers dataset.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(alzhimersdataset)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run cHooks.ipynb
try:
	#transformationPreExecutionHook()

	cautofe = TransformationMain.run(alzhimersdataset,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Argentina", "max": "USA", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "71.39", "stddev": "12.78", "min": "50", "max": "94", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EducationLevel", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.52", "stddev": "5.82", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "EducationLevel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.63", "stddev": "4.69", "min": "18.5", "max": "35.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"feature_label": "PhysicalActivityLevel", "transformation_label": "String Indexer"}], "feature": "PhysicalActivityLevel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PhysicalActivityLevel"}, {"transformationsData": [{"feature_label": "SmokingStatus", "transformation_label": "String Indexer"}], "feature": "SmokingStatus", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Current", "max": "Never", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SmokingStatus"}, {"transformationsData": [{"feature_label": "AlcoholConsumption", "transformation_label": "String Indexer"}], "feature": "AlcoholConsumption", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Never", "max": "Regularly", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AlcoholConsumption"}, {"transformationsData": [{"feature_label": "Diabetes", "transformation_label": "String Indexer"}], "feature": "Diabetes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Diabetes"}, {"transformationsData": [{"feature_label": "Hypertension", "transformation_label": "String Indexer"}], "feature": "Hypertension", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Hypertension"}, {"transformationsData": [{"feature_label": "CholesterolLevel", "transformation_label": "String Indexer"}], "feature": "CholesterolLevel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Normal", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CholesterolLevel"}, {"transformationsData": [{"feature_label": "FamilyHistoryofAlzheimers", "transformation_label": "String Indexer"}], "feature": "FamilyHistoryofAlzheimers", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FamilyHistoryofAlzheimers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CognitiveTest Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "64.61", "stddev": "20.05", "min": "30", "max": "99", "missing": "0"}, "updatedLabel": "CognitiveTest Score"}, {"transformationsData": [{"feature_label": "DepressionLevel", "transformation_label": "String Indexer"}], "feature": "DepressionLevel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepressionLevel"}, {"transformationsData": [{"feature_label": "SleepQuality", "transformation_label": "String Indexer"}], "feature": "SleepQuality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Poor", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SleepQuality"}, {"transformationsData": [{"feature_label": "DietaryHabits", "transformation_label": "String Indexer"}], "feature": "DietaryHabits", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Unhealthy", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DietaryHabits"}, {"transformationsData": [{"feature_label": "AirPollutionExposure", "transformation_label": "String Indexer"}], "feature": "AirPollutionExposure", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AirPollutionExposure"}, {"transformationsData": [{"feature_label": "MaritalStatus", "transformation_label": "String Indexer"}], "feature": "MaritalStatus", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Married", "max": "Widowed", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MaritalStatus"}, {"transformationsData": [{"feature_label": "GeneticRiskFactor", "transformation_label": "String Indexer"}], "feature": "GeneticRiskFactor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GeneticRiskFactor"}, {"transformationsData": [{"feature_label": "SocialEngagementLevel", "transformation_label": "String Indexer"}], "feature": "SocialEngagementLevel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SocialEngagementLevel"}, {"transformationsData": [{"feature_label": "StressLevels", "transformation_label": "String Indexer"}], "feature": "StressLevels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StressLevels"}, {"transformationsData": [{"feature_label": "AlzheimersDiagnosis", "transformation_label": "String Indexer"}], "feature": "AlzheimersDiagnosis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AlzheimersDiagnosis"}]}))

	#transformationPostExecutionHook(cautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run cHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(cautofe, ["Age", "EducationLevel", "BMI", "CognitiveTest Score", "Country_stringindexer", "Gender_stringindexer", "PhysicalActivityLevel_stringindexer", "SmokingStatus_stringindexer", "AlcoholConsumption_stringindexer", "Diabetes_stringindexer", "Hypertension_stringindexer", "CholesterolLevel_stringindexer", "FamilyHistoryofAlzheimers_stringindexer", "DepressionLevel_stringindexer", "SleepQuality_stringindexer", "DietaryHabits_stringindexer", "AirPollutionExposure_stringindexer", "MaritalStatus_stringindexer", "GeneticRiskFactor_stringindexer", "SocialEngagementLevel_stringindexer", "StressLevels_stringindexer"], "AlzheimersDiagnosis_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

